<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/100_coin_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import time

# CoinGecko API erişim bilgileri
COINGECKO_URL = "https://api.coingecko.com/api/v3/coins/markets"
HEADERS = {'User-Agent': 'Mozilla/5.0'}

def get_coingecko_data(symbols, vs_currency='usd'):
    """CoinGecko API üzerinden 100 kripto parayı tek istekte çeker."""
    symbol_str = ','.join(symbols)
    params = {
        'vs_currency': vs_currency,
        'ids': symbol_str.lower(),
        'order': 'market_cap_desc',
        'per_page': len(symbols),
        'page': 1,
        'sparkline': False
    }

    try:
        response = requests.get(COINGECKO_URL, params=params, headers=HEADERS, timeout=15)
        response.raise_for_status()
        data = response.json()
        if data:
            df = pd.DataFrame(data)
            df = df[['id', 'current_price', 'high_24h', 'low_24h']]
            df.rename(columns={'id': 'Coin', 'current_price': 'Close', 'high_24h': 'High', 'low_24h': 'Low'}, inplace=True)
            df['Timestamp'] = pd.to_datetime(time.time(), unit='s')
            return df
    except requests.exceptions.RequestException as e:
        print(f"API Hatası: {e}")

    return pd.DataFrame()

def calculate_devisso_trend(df, ma_period=20):
    """Basit hareketli ortalama (SMA) hesaplayarak trend sinyali üretir."""
    if df.empty:
        return df

    df['MA'] = df['Close'].rolling(ma_period, min_periods=1).mean()
    df['Signal'] = np.where(df['Close'] > df['MA'], 'BUY', 'SELL')
    return df

def scan_coins(coin_list):
    """100 coini tarayıp sinyal üretir."""
    data = get_coingecko_data(coin_list)
    if not data.empty:
        return calculate_devisso_trend(data)
    return pd.DataFrame()

def save_to_excel(df, filename="crypto_signals.xlsx"):
    """Veriyi Excel dosyasına kaydeder."""
    try:
        df.to_excel(filename, index=False)
        print(f"📊 Excel dosyası kaydedildi: {filename}")
    except Exception as e:
        print(f"Excel kaydetme hatası: {e}")

# 📌 100 Coin Listesi (Piyasa değerine göre en popülerler)
coin_list = [
    'bitcoin', 'ethereum', 'binancecoin', 'cardano', 'solana', 'xrp', 'dogecoin', 'polkadot',
    'litecoin', 'chainlink', 'uniswap', 'monero', 'avalanche-2', 'shiba-inu', 'stellar',
    'cosmos', 'algorand', 'tezos', 'aave', 'fantom', 'vechain', 'tron', 'hedera', 'theta-token',
    'elrond-erd-2', 'filecoin', 'eos', 'klaytn', 'helium', 'bitTorrent', 'dash', 'zcash',
    'iota', 'neo', 'compound', 'maker', 'enjincoin', 'chiliz', 'the-graph', 'decentraland',
    'flow', 'loopring', 'pancakeswap-token', 'quant', 'thorchain', 'bittorrent', 'arweave',
    'curve-dao-token', 'kusama', 'yearn-finance', 'harmony', 'celo', 'nexo', 'waves', 'convex-finance',
    'ankr', 'gala', 'golem', 'basic-attention-token', 'wax', 'ravencoin', 'ren', 'ocean-protocol',
    'fetch-ai', 'civic', 'storj', 'origin-protocol', 'siacoin', 'numeraire', 'sushi', 'digibyte',
    'hive', 'energy-web-token', 'mask-network', 'balancer', 'livepeer', 'fetch-ai', 'moonbeam',
    'reef', 'safepal', 'telcoin', 'chromia', 'origintrail', 'bancor', 'verasity', 'kyber-network-crystal-v2',
    'pundi-x', 'district0x', 'cartesi', 'dodo', 'superfarm', 'coti', 'rally-2', 'request', 'tellor',
    'injective-protocol', 'wanchain', 'theta-fuel', 'ultra', 'mxc', 'perpetual-protocol', 'skale',
    'litentry', 'bella-protocol'
]

# 🔄 Döngü: 100 coini her 6 dakikada bir tarar
while True:
    print(f"\n--- Tarama Zamanı: {time.ctime()} ---")
    signals_df = scan_coins(coin_list)

    if not signals_df.empty:
        print(signals_df[['Coin', 'Close', 'Signal']])  # Konsolda göster
        save_to_excel(signals_df)  # Excel olarak kaydet
    else:
        print("Sinyal yok")

    time.sleep(360)  # 6 dakikada bir tarama



--- Tarama Zamanı: Mon Feb  3 19:14:03 2025 ---
           Coin          Close Signal
0       bitcoin  100003.000000   SELL
1      ethereum    2745.450000   SELL
2        solana     212.460000   SELL
3   binancecoin     600.490000   SELL
4      dogecoin       0.274179   SELL
..          ...            ...    ...
89     verasity       0.002642   SELL
90      rally-2       0.001800   SELL
91          mxc       0.003454   SELL
92        quant       0.000018   SELL
93       fantom       0.463834   SELL

[94 rows x 3 columns]
📊 Excel dosyası kaydedildi: crypto_signals.xlsx
